In [1]:
using NetMSA
using StatsBase
using BenchmarkTools

┌ Info: Precompiling NetMSA [ff9d66d5-9e17-4192-b9f7-4feb15bf8d64]
└ @ Base loading.jl:1260


In [2]:
S1 = "abcbcdem";
S2 = "acbcfg";
S3 = "abchimn";
S4 = "abcbcjkm";

L = [S1, S2, S3, S4]

4-element Array{String,1}:
 "abcbcdem"
 "acbcfg"
 "abchimn"
 "abcbcjkm"

In [3]:
M = NetMSA.createPeerMatrix(L)

8×4 Array{Union{Missing, Char},2}:
 'a'  'a'      'a'      'a'
 'b'  'c'      'b'      'b'
 'c'  'b'      'c'      'c'
 'b'  'c'      'h'      'b'
 'c'  'f'      'i'      'c'
 'd'  'g'      'm'      'j'
 'e'  missing  'n'      'k'
 'm'  missing  missing  'm'

In [4]:
mutable struct Position 
  row::Int64
  indexes::Vector{Int64}
end

mutable struct Particle
  value::Char
  updated::Int64
  pos::Position
  best::Position
  bestvalue::Float64
  
  function Particle(value::Char, pos::Position)
    return new(value, 0, pos, pos, 0.0)
  end
end

In [33]:
function getposition(value::T, rowindex::Int64, row::SubArray{T, 1}) where (T)
  indexes = findall(i -> i == value, skipmissing(row))
  return Position(rowindex, indexes);
end

r = collect(eachrow(M))
@benchmark getposition('b', 2, r[2])


BenchmarkTools.Trial: 
  memory estimate:  416 bytes
  allocs estimate:  12
  --------------
  minimum time:     189.024 ns (0.00% GC)
  median time:      228.659 ns (0.00% GC)
  mean time:        342.172 ns (20.56% GC)
  maximum time:     22.723 μs (95.02% GC)
  --------------
  samples:          10000
  evals/sample:     656

In [35]:
function getposition(value::T, rowindex::Int64, row::Array{T, 1}) where (T)
  indexes = findall(i -> i == value, skipmissing(row))
  return Position(rowindex, indexes);
end

@benchmark getposition('b', 2, M[2, :])

BenchmarkTools.Trial: 
  memory estimate:  544 bytes
  allocs estimate:  14
  --------------
  minimum time:     569.892 ns (0.00% GC)
  median time:      658.065 ns (0.00% GC)
  mean time:        887.365 ns (12.01% GC)
  maximum time:     60.381 μs (97.84% GC)
  --------------
  samples:          10000
  evals/sample:     186

In [36]:
function getposition(value::T, rowindex::Int64, matrix::Matrix{T}) where (T)
  indexes = findall(i -> i == value, skipmissing(matrix[rowindex, :]))
  return Position(rowindex, indexes);
end

@benchmark getposition('b', 2, M)

BenchmarkTools.Trial: 
  memory estimate:  528 bytes
  allocs estimate:  13
  --------------
  minimum time:     204.665 ns (0.00% GC)
  median time:      242.000 ns (0.00% GC)
  mean time:        385.117 ns (25.78% GC)
  maximum time:     19.744 μs (98.72% GC)
  --------------
  samples:          10000
  evals/sample:     600

In [5]:
function getposition(index::Int64, row, value)
  indexes = findall(i -> i == value, skipmissing(row))
  return Position(index, indexes);
end

function mostfrequent(row)
  counts = countmap(row);
  delete!(counts, '-');
#   println(row)
#   println(counts)
  max = isempty(counts) ? 0 : findmax(counts);
  return max;
end

function aligned(row)::Bool
  row = Set(row)
  return (length(row) == 1 && !(missing in row)) || (length(row) == 2 && ('-' in row || missing in row))
end

function full(row)::Bool
  return length(Set(row)) == 1 && !(missing in Set(row))
end

function weight(row; w1=0.25, w2=0.5, w3=1.0)
  if full(row)
    return w3;
  end
  
  c = length(row);
  if c == 0
    return 0;
  end
  max = mostfrequent(row)[1];
  
  if aligned(row)
    return w2 * max / c;
  else
    x = max <= 1 ? 0 : max;
    return w1 * x / c;
  end
end

function objective(M, rowindex::Int; endindex::Int=0)
  weights = sum(weight.(eachrow(M[rowindex:end, :])))
  C = mostfrequent(M[rowindex, :])[1];
  A = sum(aligned.(eachrow(M))[rowindex:end])
  
  endindex = endindex == 0 ? size(M)[1] : endindex;
  if endindex > size(M)[1]
    throw(ArgumentError("endind exceeds the matrix size"));
  end
  counts = countmap(M[rowindex:endindex, :]);
  Gaps = get(counts, '-', 0);
  
  
  
#   println(A)
#   println(C)
#   println(Gaps)
#   println(weights)
#   println(sum(weights))
  
  return weights * (A * C)/(1 + Gaps)
end

function createswarm(rowindex::Int64, row)
  unique = Set(skipmissing(row))
  swarm = Vector{Particle}(undef, length(unique))
  for (i, c) in enumerate(unique)
    swarm[i] = Particle(c, getposition(rowindex, row, c))
  end
  return swarm
end

function criteria3(p::Particle, M, newindex)
#   display(newindex)
#   display(M[newindex, :])
#   display(length(p.pos.indexes) != length(getposition(newindex, M[newindex, :], p.value).indexes))
  return length(p.pos.indexes) != length(getposition(newindex, M[newindex, :], p.value).indexes)
end

function criteria2(p::Particle)
  return p.updated > 6;
end

function stopcriteria(p::Particle, M, t)
  c3 = criteria3(p, M, t);
  c2 = criteria2(p);
  if c3
    display("Terminating cause of criteria 3")
  elseif  c2
    display("Terminating cause of criteria 2")
  end
  return c3 || c2;
end

function remove_missing_rows(M)
  return M[[length(Set(skipmissing(r))) != 0 for r in eachrow(M)], :]
end

remove_missing_rows (generic function with 1 method)

In [9]:
function flydown(p, M; stride=1)
  notpcols = setdiff(collect(1:size(M, 2)), p.pos.indexes)
  colsize = size(M, 2)
  pos = p.pos
  newrows = fill('-', (stride, colsize))
  M = vcat(M[1:pos.row - 1, :], reshape(newrows, stride, colsize), M[pos.row:end, :])
  display(M)
  for i in collect(pos.row+stride:size(M, 1))
    M[i-stride,notpcols] = M[i, notpcols]
    M[i, notpcols] .= missing
  end
#   display(M)
  M = remove_missing_rows(M)
  return M
end

N = copy(M)
c = 'c'
p = Particle(c, getposition(2, N[2, :], c))
N = flydown(p, N, stride=3)

11×4 Array{Union{Missing, Char},2}:
 'a'  'a'      'a'      'a'
 '-'  '-'      '-'      '-'
 '-'  '-'      '-'      '-'
 '-'  '-'      '-'      '-'
 'b'  'c'      'b'      'b'
 'c'  'b'      'c'      'c'
 'b'  'c'      'h'      'b'
 'c'  'f'      'i'      'c'
 'd'  'g'      'm'      'j'
 'e'  missing  'n'      'k'
 'm'  missing  missing  'm'

9×4 Array{Union{Missing, Char},2}:
 'a'      'a'  'a'      'a'
 'b'      '-'  'b'      'b'
 'c'      '-'  'c'      'c'
 'b'      '-'  'h'      'b'
 'c'      'c'  'i'      'c'
 'd'      'b'  'm'      'j'
 'e'      'c'  'n'      'k'
 'm'      'f'  missing  'm'
 missing  'g'  missing  missing

In [ ]:
M

In [12]:
function rowalignment(r, M)
  row = M[r, :];
#   display(row);
#   println(aligned(row))
  if aligned(row)
#     println("aligned");
    return nothing;
  end
  
  swarm = createswarm(r, row);
  
  gₒ = g = swarm[1];
  gₒvalue = gvalue = objective(M, r, endindex=r);
  cols = size(row, 1)
  
  for p in swarm
    
    t = r;
    N = copy(M);
    
    p.bestvalue = objective(M, r, endindex=t)
#     display("Aligning $p");
#     display(N)
#     display(p)
    
    missingp = setdiff(collect(1:size(N, 2)), p.pos.indexes)
    maxlen = maximum([length(collect(skipmissing(col))) for col in eachcol(N[:, missingp])])
    criteria1 = maxlen;
    
#     display(criteria1)
    
    while stopcriteria(p, N, t) != true && t < criteria1
#       display(stopcriteria(p, N, t) != true)
      t += 1;
      p.updated += 1;
     
      N = flydown(p, N);
#       display(N)
      display(p)
      score = objective(N, r);
#       display(score);
#       display(p.bestvalue);
      if score > p.bestvalue
        p.bestvalue = score;
        p.updated = 0;
      end
      
      if score > gvalue
        gvalue = score;
        g = deepcopy(p);
        g.best = getposition(t, N[t, :], p.value);
        g.bestvalue = score;
      end
      
      
#       display(p)
      
    end
  end
  
  if gvalue == gₒvalue
    return nothing;
  end
  return g;
end

rowalignment (generic function with 1 method)

In [13]:
N = copy(M)
g = rowalignment(2, N)
N = flydown(g, N, stride=g.best.row - g.pos.row)
objective(N, 2)

9×4 Array{Union{Missing, Char},2}:
 'a'  'a'      'a'      'a'
 '-'  '-'      '-'      '-'
 'b'  'c'      'b'      'b'
 'c'  'b'      'c'      'c'
 'b'  'c'      'h'      'b'
 'c'  'f'      'i'      'c'
 'd'  'g'      'm'      'j'
 'e'  missing  'n'      'k'
 'm'  missing  missing  'm'

Particle('c', 1, Position(2, [2]), Position(2, [2]), 2.625)

"Terminating cause of criteria 3"

9×4 Array{Union{Missing, Char},2}:
 'a'  'a'      'a'      'a'
 '-'  '-'      '-'      '-'
 'b'  'c'      'b'      'b'
 'c'  'b'      'c'      'c'
 'b'  'c'      'h'      'b'
 'c'  'f'      'i'      'c'
 'd'  'g'      'm'      'j'
 'e'  missing  'n'      'k'
 'm'  missing  missing  'm'

Particle('b', 1, Position(2, [1, 3, 4]), Position(2, [1, 3, 4]), 2.625)

"Terminating cause of criteria 3"

9×4 Array{Union{Missing, Char},2}:
 'a'  'a'      'a'      'a'
 '-'  '-'      '-'      '-'
 'b'  'c'      'b'      'b'
 'c'  'b'      'c'      'c'
 'b'  'c'      'h'      'b'
 'c'  'f'      'i'      'c'
 'd'  'g'      'm'      'j'
 'e'  missing  'n'      'k'
 'm'  missing  missing  'm'

9.0

In [14]:
g = rowalignment(3, N)
# N = flydown(g, N, stride=g.best.row - g.pos.row)

In [15]:
g = rowalignment(4, N)
N = flydown(g, N, stride=g.best.row - g.pos.row)

9×4 Array{Union{Missing, Char},2}:
 'a'  'a'      'a'      'a'
 'b'  '-'      'b'      'b'
 'c'  'c'      'c'      'c'
 '-'  '-'      '-'      '-'
 'b'  'b'      'h'      'b'
 'c'  'c'      'i'      'c'
 'd'  'f'      'm'      'j'
 'e'  'g'      'n'      'k'
 'm'  missing  missing  'm'

Particle('h', 1, Position(4, [3]), Position(4, [3]), 1.875)

9×4 Array{Union{Missing, Char},2}:
 'a'  'a'      'a'  'a'
 'b'  '-'      'b'  'b'
 'c'  'c'      'c'  'c'
 '-'  '-'      '-'  '-'
 'b'  'b'      '-'  'b'
 'c'  'c'      'h'  'c'
 'd'  'f'      'i'  'j'
 'e'  'g'      'm'  'k'
 'm'  missing  'n'  'm'

Particle('h', 2, Position(4, [3]), Position(4, [3]), 1.875)

10×4 Array{Union{Missing, Char},2}:
 'a'      'a'      'a'  'a'
 'b'      '-'      'b'  'b'
 'c'      'c'      'c'  'c'
 '-'      '-'      '-'  '-'
 'b'      'b'      '-'  'b'
 'c'      'c'      '-'  'c'
 'd'      'f'      'h'  'j'
 'e'      'g'      'i'  'k'
 'm'      missing  'm'  'm'
 missing  missing  'n'  missing

...

Particle('b', 2, Position(4, [1, 2, 4]), Position(4, [1, 2, 4]), 1.875)

11×4 Array{Union{Missing, Char},2}:
 'a'  'a'      'a'      'a'
 'b'  '-'      'b'      'b'
 'c'  'c'      'c'      'c'
 '-'  '-'      '-'      '-'
 '-'  '-'      'h'      '-'
 '-'  '-'      'i'      '-'
 'b'  'b'      'm'      'b'
 'c'  'c'      'n'      'c'
 'd'  'f'      missing  'j'
 'e'  'g'      missing  'k'
 'm'  missing  missing  'm'

Particle('b', 3, Position(4, [1, 2, 4]), Position(4, [1, 2, 4]), 1.875)

10×4 Array{Union{Missing, Char},2}:
 'a'  'a'      'a'      'a'
 'b'  '-'      'b'      'b'
 'c'  'c'      'c'      'c'
 '-'  '-'      '-'      '-'
 '-'  '-'      '-'      '-'
 'b'  'b'      'h'      'b'
 'c'  'c'      'i'      'c'
 'd'  'f'      'm'      'j'
 'e'  'g'      'n'      'k'
 'm'  missing  missing  'm'

9×4 Array{Union{Missing, Char},2}:
 'a'      'a'      'a'  'a'
 'b'      '-'      'b'  'b'
 'c'      'c'      'c'  'c'
 'b'      'b'      '-'  'b'
 'c'      'c'      '-'  'c'
 'd'      'f'      'h'  'j'
 'e'      'g'      'i'  'k'
 'm'      missing  'm'  'm'
 missing  missing  'n'  missing

In [16]:
display(N)
objective(N, 4)

9×4 Array{Union{Missing, Char},2}:
 'a'      'a'      'a'  'a'
 'b'      '-'      'b'  'b'
 'c'      'c'      'c'  'c'
 'b'      'b'      '-'  'b'
 'c'      'c'      '-'  'c'
 'd'      'f'      'h'  'j'
 'e'      'g'      'i'  'k'
 'm'      missing  'm'  'm'
 missing  missing  'n'  missing

6.0

In [8]:
function matrixalignment(M)
  for (index, row) in enumerate(eachrow(M))
  #   println("$index: $row")
    g = rowalignment(index, M) 
    if !isnothing(g)
      M = flydown(g, M, stride=g.best.row - g.pos.row)
    end
  end
  replace!(M, missing => '-')
end
matrixalignment(M)

9×4 Array{Union{Missing, Char},2}:
 'a'  'a'      'a'      'a'
 '-'  '-'      '-'      '-'
 'b'  'c'      'b'      'b'
 'c'  'b'      'c'      'c'
 'b'  'c'      'h'      'b'
 'c'  'f'      'i'      'c'
 'd'  'g'      'm'      'j'
 'e'  missing  'n'      'k'
 'm'  missing  missing  'm'

9×4 Array{Union{Missing, Char},2}:
 'a'      'a'      'a'      'a'
 'b'      '-'      'b'      'b'
 'c'      'c'      'c'      'c'
 'b'      'b'      'h'      'b'
 'c'      'c'      'i'      'c'
 'd'      'f'      'm'      'j'
 'e'      'g'      'n'      'k'
 'm'      missing  missing  'm'
 missing  missing  missing  missing

Particle('c', 1, Position(2, [2]), Position(2, [2]), 2.625)

9.0

"Terminating cause of criteria 3"

...

10×4 Array{Union{Missing, Char},2}:
 'a'      'a'      'a'  'a'
 'b'      '-'      'b'  'b'
 'c'      'c'      'c'  'c'
 'b'      'b'      '-'  'b'
 'c'      'c'      '-'  'c'
 'd'      'f'      'h'  'j'
 '-'      '-'      '-'  '-'
 'e'      'g'      'i'  '-'
 'm'      missing  'm'  'k'
 missing  missing  'n'  'm'

10×4 Array{Union{Missing, Char},2}:
 'a'      'a'      'a'      'a'
 'b'      '-'      'b'      'b'
 'c'      'c'      'c'      'c'
 'b'      'b'      '-'      'b'
 'c'      'c'      '-'      'c'
 'd'      'f'      'h'      'j'
 'e'      'g'      'i'      '-'
 'm'      missing  'm'      '-'
 missing  missing  'n'      'k'
 missing  missing  missing  'm'

Particle('k', 2, Position(7, [4]), Position(7, [4]), 1.5)

0.20833333333333334

9×4 Array{Union{Missing, Char},2}:
 'a'  'a'  'a'  'a'
 'b'  '-'  'b'  'b'
 'c'  'c'  'c'  'c'
 'b'  'b'  '-'  'b'
 'c'  'c'  '-'  'c'
 'd'  'f'  'h'  'j'
 'e'  'g'  'i'  'k'
 'm'  '-'  'm'  'm'
 '-'  '-'  'n'  '-'